In [1]:
import warnings
warnings.filterwarnings('ignore')

### Сентимент-анализ отзывов (простая версия)

In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

Для начала загрузим данные и посмотрим на их общий вид и размерность

In [17]:
train = pd.read_csv('simplesentiment/products_sentiment_train.tsv', sep='\t', names = ['text','label'])
test = pd.read_csv('simplesentiment/products_sentiment_test.tsv', sep='\t', index_col = 'Id')

y_train = train.label

In [18]:
# Пример нескольких отзывов
pd.set_option('max_colwidth', 300)
test.head()

text
Id                                                                                                                                                                                              
0   so , why the small digital elph , rather than one of the other cameras with better resolution or picture quality ? size [ + 2 ] # # because , unless it 's small , i won 't cary it around .
1                                                                                 3/4 way through the first disk we played on it ( naturally on 31 days after purchase ) the dvd player froze . 
2                                                                                                                                            better for the zen micro is outlook compatibility .
3                                                                                                                                                6 . play gameboy color games on it with goboy .
4                                                                                                                          likewise , i 've heard norton 2004 professional version is fine too .

Заменим в каждом тексте сокращения  've 't на have и not соответственно.

In [19]:
def preprocess(text):
    text = text.replace("'ve", "have")
    #text = text.replace("won 't", "will not")
    return text.replace(" 't", " not")

train['x'] = train.text.apply(preprocess)
test['x'] = test.text.apply(preprocess)


Посчитаем длину каждого отзыва, сколько в нем слов и сколько знаков. Результат запишем в наши даные и проанализируем.

In [22]:
train["length_message"] = train["x"].apply(lambda s: len(s.split()))
train["number_caracters"] = train["x"].apply(lambda s: len(s))

In [23]:
test["lenth_message"] = test["x"].apply(lambda s: len(s.split()))
test["number_caracters"] = test["x"].apply(lambda s: len(s))

In [26]:
train.describe()

label  length_message  number_caracters
count  2000.000000      2000.00000       2000.000000
mean      0.637000        19.75600         95.842500
std       0.480985        11.55661         57.274924
min       0.000000         2.00000          8.000000
25%       0.000000        11.00000         55.000000
50%       1.000000        18.00000         84.000000
75%       1.000000        25.00000        124.000000
max       1.000000        92.00000        495.000000

In [24]:
test.describe()

lenth_message  number_caracters
count      500.00000        500.000000
mean        20.43400         99.480000
std         12.32418         61.438742
min          2.00000         11.000000
25%         12.00000         56.000000
50%         17.50000         85.000000
75%         25.00000        124.250000
max        106.00000        536.000000

Видим, что в среднем отзывы состоят из 20ти слов и около 100 символов, тоесть сообщения очень короткие. В дальнейшем избежим исользование методов, что сократят ещё больше их размер, например удаление стоп-слов. Также оставим все символы и пунктуацию, так как некоторые знаки помогают определить тональность отзыва.

Загрузим все необходимые библиотеки:

In [43]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import model_selection

from sklearn.decomposition import NMF, TruncatedSVD
from nltk import word_tokenize  
from nltk.stem.snowball import PorterStemmer

import re # For regular expressions

Для начала выберем класификатор, который лучше работает для наших даных. Будем выбирать среди логистической регресии, метода опорных векторов, градиентного спуска и наивного баеса. Для векторизации векторов воспользуемся TF-IDF.

In [46]:
for classifier in [LogisticRegression, LinearSVC, SGDClassifier, BernoulliNB, ComplementNB, MultinomialNB]:
    clf_pipeline = Pipeline(
            [("vectorizer", TfidfVectorizer()),
            ("classifier", classifier())]
        )
    print(classifier)
    print('\n')
    print(cross_val_score(clf_pipeline, train.x, y_train, scoring='accuracy').mean())

<class 'sklearn.linear_model.logistic.LogisticRegression'>


0.7580061320691005
<class 'sklearn.svm.classes.LinearSVC'>


0.7664988826907869
<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'>


0.7610023816920369
<class 'sklearn.naive_bayes.BernoulliNB'>


0.737511124317721
<class 'sklearn.naive_bayes.ComplementNB'>


0.7610046328187258
<class 'sklearn.naive_bayes.MultinomialNB'>


0.6985036010523267


Наилучшие результаты у метода опорных векторов (SVM). Этот метод хорошо зарекомендовал себя для подобных задач. В дальнейшем будем использовать его. А  сейчас попробуем подобрать оптимальные параметры для нашей модели.

Для начала посмотрим на параметр ngram_range, что определяет сколько н-грам будет использоватся. С наборами (1,4), (1,5),(4,7), (4,8),(4,9),(4,10),(4,11),(4,12) посмотрим на н-грамы по буквам.

In [35]:
for ngram in [(1,4), (1,5),(4,7), (4,8),(4,9),(4,10),(4,11),(4,12)]:
    clf_pipeline = Pipeline(
            [("vectorizer", TfidfVectorizer(ngram_range = ngram, analyzer = 'char')),
            ("classifier", LinearSVC())]
        )
    print(ngram)
    print('\n')
    print(cross_val_score(clf_pipeline, train.x, y_train, scoring='accuracy').mean())

(1, 4)


0.77349988669329
(1, 5)


0.7855006430718575
(4, 7)


0.7915036475756115
(4, 8)


0.7920011465738602
(4, 9)


0.7925001463232347
(4, 10)


0.789003396199798
(4, 11)


0.788005396701049
(4, 12)


0.7865046455751105


Видим, что наилучший результат получаеться, если выбирать наборы от 4- до 9-грам по буквам, что соответствует средним словам или коротким словосочетаниям.

А что если выбрать словесные н-граммы? Посмотрим на результаты если использовать разные комбинации униграмм, биграмм, триграмм.

In [37]:
for ngram in [(1,1), (1,2),(2,2), (1,3),(2,3),(3,3)]:
    clf_pipeline = Pipeline(
            [("vectorizer", TfidfVectorizer(ngram_range = ngram, analyzer = 'word')),
            ("classifier", LinearSVC())]
        )
    print(ngram)
    print('\n')
    print(cross_val_score(clf_pipeline, train.x, y_train, scoring='accuracy').mean())

(1, 1)


0.7664988826907869
(1, 2)


0.7864986425706065
(2, 2)


0.7394981188084637
(1, 3)


0.7869991430711071
(2, 3)


0.7344991168079624
(3, 3)


0.6789960875418147


Использование только триграмм значительно ухудшает качество предсказания. Немного лучше ситуация для биграмм и триграмм вместе или только биграмм. В нашем случае оптимально использовать все три вида н-грамм. Также заметим, что качество векторизации немного лучше если выбирать побуквенные граммы, а не словесные.

Попробуем улучшить модель воспользовавшись одной из практик предобработки текста стеммированием. Данный метод хорошо  подходит к английскому языку.

In [38]:
stemmer = PorterStemmer()
analyzer = TfidfVectorizer().build_analyzer()


def stemmed(text):
    return (stemmer.stem(word) for word in analyzer(preprocess(text)))

Проверим как работает векторизация с уныграмамми, биграмамми и триграмамми на стемах.

In [60]:
for ngram in [(1,1), (1,2),(2,2), (1,3),(2,3),(3,3)]:
    clf_pipeline = Pipeline(
            [("vectorizer", TfidfVectorizer(ngram_range = ngram, analyzer = stemmed)),
            ("classifier", LinearSVC())]
        )
    print(ngram)
    print('\n')
    print(cross_val_score(clf_pipeline, X_train, y_train, scoring='accuracy').mean())

(1, 1)


0.7715008861935398
(1, 2)


0.7715008861935398
(2, 2)


0.7715008861935398
(1, 3)


0.7715008861935398
(2, 3)


0.7715008861935398
(3, 3)


0.7715008861935398


Получили одинаковое число для всех вариантов.

А сейчас попытаемся соединить все лучшие варианты в один и проверим как работает на ней модель. Для этого воспользуемся методом FeatureUnion с scikit-learn

In [41]:
union = FeatureUnion([("word13", TfidfVectorizer(ngram_range=(1,3), analyzer='word')),
                      ("stem13", TfidfVectorizer(ngram_range=(1,3), analyzer=stemmed)),
                      ("char14", TfidfVectorizer(ngram_range=(1,5), analyzer='char')),
                      ("char410", TfidfVectorizer(ngram_range=(4,10), analyzer='char')),
                      
                     ])



Используем грид сьорч для подбора параметров для метода опорных векторов.


In [40]:
# проверим какие параметры есть в LinearSVC
LinearSVC().get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'loss', 'max_iter', 'multi_class', 'penalty', 'random_state', 'tol', 'verbose'])

In [29]:
parameters_grid = {
    'classifier__C': list(np.arange(0.05, 1.0, 0.05)),
}

cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 0)

In [30]:
clf_pipeline = Pipeline(
            [("vectorizer", union),
            ("classifier", LinearSVC(fit_intercept = False, dual  = False))]
        )

grid_cv = model_selection.GridSearchCV(clf_pipeline, parameters_grid, scoring = 'accuracy', cv = cv)

Обучение может занять время.

In [31]:
%%time
grid_cv.fit(train.x, y_train)

Wall time: 38min 26s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vectorizer',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('word13',
                                                                        TfidfVectorizer(analyzer='word',
                                                                                        binary=False,
                                                                                        decode_error='strict',
                                                                                        dtype=<class 'numpy.float64'>,
                                                                                        encoding='utf-8',
                                                   

In [32]:
grid_cv.best_estimator_

Pipeline(memory=None,
         steps=[('vectorizer',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('word13',
                                                 TfidfVectorizer(analyzer='word',
                                                                 binary=False,
                                                                 decode_error='strict',
                                                                 dtype=<class 'numpy.float64'>,
                                                                 encoding='utf-8',
                                                                 input='content',
                                                                 lowercase=True,
                                                                 max_df=1.0,
                                                                 max_features=None,
                                                                 min_df=1,
                  

Подобрали С = 0.5. Теперь можно посмотреть на оценку с выбраным векторизатором и класификатором с выбраными параметрами.

In [42]:
clf_pipeline = Pipeline(
            [("vectorizer", union),
            ("classifier", LinearSVC(C=0.5, fit_intercept = False, dual = False))]
        )
print(cross_val_score(clf_pipeline, train.x, y_train, scoring='accuracy').mean())

0.7944998971985479


Оценка улучшилась очень незначительно. Но это лучший результат со всех расмотреных. Веберем даную модель для предсказания на тестовой выборке, а также сохраним результат в файл для отправки на соревнование.

In [34]:
# Обучим классификатор на всех размененных данных
clf_pipeline.fit(train.x, y_train)
test['y'] = clf_pipeline.predict(test.x)


In [35]:
test.head()

text  \
Id                                                                                                                                                                                                 
0   so , why the small digital elph , rather than one of the other cameras with better resolution or picture quality ? size [ + 2 ] # # because , unless it 's small , i won 't cary it around .   
1                                                                                 3/4 way through the first disk we played on it ( naturally on 31 days after purchase ) the dvd player froze .    
2                                                                                                                                            better for the zen micro is outlook compatibility .   
3                                                                                                                                                6 . play gameboy color games on it with goboy .   
4                                                                                                                          likewise , i 've heard norton 2004 professional version is fine too .   

                                                                                                                                                                                                x  \
Id                                                                                                                                                                                                  
0   so , why the small digital elph , rather than one of the other cameras with better resolution or picture quality ? size [ + 2 ] # # because , unless it 's small , i won not cary it around .   
1                                                                                  3/4 way through the first disk we played on it ( naturally on 31 days after purchase ) the dvd player froze .    
2                                                                                                                                             better for the zen micro is outlook compatibility .   
3                                                                                                                                                 6 . play gameboy color games on it with goboy .   
4                                                                                                                          likewise , i have heard norton 2004 professional version is fine too .   

    lenth_message  number_caracters  y  
Id                                      
0              43               189  1  
1              23               110  0  
2               9                51  1  
3              11                47  1  
4              13                70  0

In [36]:
# Запишем в файл решение для загрузки на Kaggle
test[['y']].to_csv('product-reviews-sentiment-analysis-light1.csv')